In [1]:
!pip install -U rectools -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.9.0 which is incompatible.


In [35]:
import rectools
from rectools.dataset import Dataset
from rectools import Columns
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

In [74]:
data = pd.read_parquet('/kaggle/input/vseros-recsys/train_data.pq')
sub = pd.read_csv('/kaggle/input/vseros-recsys/sample_submission.csv')

data = data.rename({'date': Columns.Datetime}, axis=1)
data[Columns.Weight] = 1

In [8]:
tr_cf, oof_cf = data[data['datetime'] < 42], data[data['datetime'] >= 42]

In [23]:
users_in_oof = data[data['datetime'] >= 42]['user_id'].unique()
tr_cf = data[(data['datetime'] < 42) | (data['user_id'].isin(users_in_oof) & (data['datetime'] == data.groupby('user_id')['datetime'].transform('min')))]
oof_cf = data[(data['datetime'] >= 42) & ~(data['datetime'] == data.groupby('user_id')['datetime'].transform('min'))]

In [24]:
tr_cf.loc[tr_cf['datetime'] < 38, 'weight'] = 0.1

In [75]:
data.loc[data['datetime'] < 44, 'weight'] = 0.1

In [76]:
from datetime import datetime, timedelta

# Базовая дата - 10 января 2020
base_date = datetime(2020, 1, 10)

# Преобразование чисел 0-46 в даты
dates = {}
for i in range(47):
    current_date = base_date + timedelta(days=i)
    dates[i] = current_date.strftime("%d.%m.%Y")

In [77]:
data[Columns.Datetime] = data[Columns.Datetime].map(dates)
data[Columns.Datetime] = pd.to_datetime(data['datetime'])

In [27]:
tr_cf[Columns.Datetime] = tr_cf[Columns.Datetime].map(dates)
tr_cf[Columns.Datetime] = pd.to_datetime(tr_cf['datetime'])

In [28]:
cf_ds = Dataset.construct(tr_cf)

In [78]:
ds = Dataset.construct(data)

In [100]:
from rectools.models import PureSVDModel, HSTUModel, ImplicitItemKNNWrapperModel
from implicit.nearest_neighbours import ItemItemRecommender, TFIDFRecommender, BM25Recommender, CosineRecommender

In [58]:
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel, PopularModel
from rectools import Columns
from rectools.dataset import Dataset
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

In [121]:
def make_base_model(factors: int, regularization: float, alpha: float, iterations, fit_features_together: bool=False):
    return ImplicitALSWrapperModel(
        AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            alpha=alpha,
            random_state=42,
            use_gpu=False,
            num_threads = 4,
            iterations=iterations,
        calculate_training_loss=True),
        fit_features_together = fit_features_together, verbose=1
        )

In [122]:
model = make_base_model(32, 25, 100, 15)
model.fit(ds)

  0%|          | 0/15 [00:00<?, ?it/s]

In [79]:
model = ImplicitItemKNNWrapperModel(ItemItemRecommender(K=40))
model.fit(ds)

In [101]:
model = ImplicitItemKNNWrapperModel(CosineRecommender(K=40))
model.fit(ds)

In [81]:
pop_model = PopularModel('sum_weight')
pop_model.fit(ds)

In [115]:
good_items = data.loc[data['datetime'] > pd.to_datetime(dates[40]), 'item_id'].unique()

In [123]:
recos = model.recommend(
                users=sub['user_id'].unique(),
                dataset=ds,
                k=20,
                filter_viewed=False,
                items_to_recommend=good_items,
            )

In [86]:
pop_recos = pop_model.recommend(
                users=sub['user_id'].unique(),
                dataset=ds,
                k=20,
                filter_viewed=False
            )

In [111]:
sum(recos['user_id'].value_counts() != 20)

0

In [87]:
sum(pop_recos['user_id'].value_counts() != 20)

0

In [92]:
import pandas as pd

def combine_submissions(sub1, sub2):
    result = []
    sub1_groups = sub1.groupby('user_id')['item_id'].apply(list)
    sub2_groups = sub2.groupby('user_id')['item_id'].apply(list)
    
    for user_id in tqdm(sub1_groups.index):
        items1 = sub1_groups[user_id]
        items2 = sub2_groups.get(user_id, [])
        seen = set(items1)
        
        for item in items2:
            if len(items1) >= 20:
                break
            if item not in seen:
                items1.append(item)
                seen.add(item)
        
        for item_id in items1:
            result.append((user_id, item_id))
    
    return pd.DataFrame(result, columns=['user_id', 'item_id'])

In [93]:
comb_recos = combine_submissions(recos, pop_recos)

  0%|          | 0/293230 [00:00<?, ?it/s]

In [97]:
comb_recos.to_csv('item-item-knn-k40-and-pop.csv', index=False)

In [118]:
recos[['user_id', 'item_id']].to_csv('als-20-20-15.csv', index=False)

In [117]:
recos

user_id  item_id     score  rank
0         247446     1030  0.301157     1
1         247446      310  0.234731     2
2         247446     1439  0.214721     3
3         247446     1028  0.177166     4
4         247446    57014  0.117375     5
...          ...      ...       ...   ...
5864595   244238      229  0.184195    16
5864596   244238      298  0.179054    17
5864597   244238    71800  0.175651    18
5864598   244238     1500  0.171815    19
5864599   244238      592  0.144033    20

[5864600 rows x 4 columns]

In [69]:
def calculate_map_at_20_optimized(ground_truth_df, submission_df):
    """
    Оптимизированная версия функции расчета mAP@20
    """
    # Создаем словарь с реальными взаимодействиями
    ground_truth = ground_truth_df.groupby('user_id')['item_id'].apply(set).to_dict()
    
    # Группируем предсказания
    user_predictions = submission_df.groupby('user_id')['item_id'].apply(list).to_dict()
    
    average_precisions = []
    
    for user_id, pred_items in tqdm(user_predictions.items()):
        if user_id not in ground_truth:
            continue
            
        true_items = ground_truth[user_id]
        if not true_items:
            continue
        
        # Ограничиваем предсказания 20 элементами
        pred_items = pred_items[:20]
        
        # Вычисляем relevancy для каждого предсказанного товара
        relevancy = [1 if item in true_items else 0 for item in pred_items]
        
        # Вычисляем cumulative sum для precision@k
        cumulative_correct = np.cumsum(relevancy)
        positions = np.arange(1, len(relevancy) + 1)
        precisions = cumulative_correct / positions
        
        # Умножаем только на relevancy, чтобы учесть только релевантные позиции
        weighted_precisions = precisions * relevancy
        
        # Average Precision = сумма precisions для релевантных товаров / min(количество релевантных, 20)
        ap = np.sum(weighted_precisions) / min(len(true_items), 20)
        average_precisions.append(ap)
    
    return np.mean(average_precisions) if average_precisions else 0.0

In [73]:
calculate_map_at_20_optimized(oof_cf, recos)

  0%|          | 0/195412 [00:00<?, ?it/s]

0.025398005275387266

In [43]:
calculate_map_at_20_optimized(oof_cf, recos)

  0%|          | 0/195412 [00:00<?, ?it/s]

0.08965580387978915

In [52]:
recos['score_rank'] = list(range(1, 101)[::-1]) * (len(oof_cf['user_id'].unique()))

ValueError: Length of values (19541200) does not match length of index (14095117)

In [53]:
len(list(range(1, 101)[::-1]) * len([1]))

100

In [55]:
recos['user_id'].value_counts()

user_id
240466     100
1424899    100
1710593    100
215565     100
215564     100
          ... 
896102       1
1420387      1
362086       1
947913       1
234462       1
Name: count, Length: 195412, dtype: int64

ВУУВЩВЫ=М=Ч

In [4]:
from tqdm.auto import tqdm

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
train = pd.read_parquet('/kaggle/input/vseros-recsys/train_data.pq')
sample_sub = pd.read_csv('/kaggle/input/vseros-recsys/sample_submission.csv')

print(f"Размер train: {train.shape}")
max_date = train['date'].max()
test_users = sample_sub['user_id'].unique()
print(f"Тестовых пользователей: {len(test_users)}")

def fast_hybrid_recommendations(train_data, user_list, top_n=20):
    """Ускоренная гибридная система рекомендаций"""
    
    max_date = train_data['date'].max()
    
    # 1. Популярные товары за последнюю неделю (самый быстрый метод)
    last_week_mask = train_data['date'] >= (max_date - 6)
    popular_items = train_data[last_week_mask]['item_id'].value_counts().head(top_n * 3).index.tolist()
    
    # 2. User-item матрица для быстрого доступа
    user_recent_items = train_data[last_week_mask].groupby('user_id')['item_id'].apply(set).to_dict()
    
    # 3. Item-user матрица для collaborative filtering
    item_users_dict = train_data[last_week_mask].groupby('item_id')['user_id'].apply(set).to_dict()
    
    # 4. Глобальная история пользователей
    user_all_items = train_data.groupby('user_id')['item_id'].apply(set).to_dict()
    
    final_recommendations = {}
    
    for user in tqdm(user_list):
        user_recs = []
        user_history = user_all_items.get(user, set())
        
        # A. Персональные рекомендации через item-based CF
        if user in user_recent_items:
            recent_user_items = user_recent_items[user]
            
            # Быстрый item-based CF через пересечения
            candidate_scores = defaultdict(int)
            for item in recent_user_items:
                if item in item_users_dict:
                    for other_user in item_users_dict[item]:
                        if other_user != user and other_user in user_recent_items:
                            for other_item in user_recent_items[other_user]:
                                if other_item not in user_history:
                                    candidate_scores[other_item] += 1
            
            # Сортируем и берем топ
            personal_recs = sorted(candidate_scores.keys(), key=lambda x: candidate_scores[x], reverse=True)[:15]
            user_recs.extend(personal_recs)
        
        # B. Дополняем популярными товарами
        for item in popular_items:
            if item not in user_history and item not in user_recs and len(user_recs) < top_n:
                user_recs.append(item)
        
        # C. Гарантируем 20 рекомендаций
        if len(user_recs) < top_n:
            global_popular = train_data['item_id'].value_counts().head(50).index.tolist()
            for item in global_popular:
                if item not in user_history and item not in user_recs and len(user_recs) < top_n:
                    user_recs.append(item)
        
        final_recommendations[user] = user_recs[:top_n]
    
    return final_recommendations

print("Генерация ускоренных рекомендаций...")
recommendations = fast_hybrid_recommendations(train, test_users)

# Создаем сабмит
submission_rows = []
for user, items in recommendations.items():
    for item in items:
        submission_rows.append({'user_id': user, 'item_id': item})

pd.DataFrame(submission_rows).to_csv('fast_submission.csv', index=False)
print("Ускоренный сабмит сохранен!")

Размер train: (8777975, 3)
Тестовых пользователей: 293230
Генерация ускоренных рекомендаций...


  0%|          | 0/293230 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_37/2634023071.py", line 75, in <cell line: 0>
    recommendations = fast_hybrid_recommendations(train, test_users)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_37/2634023071.py", line None, in fast_hybrid_recommendations
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib

In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pyarrow.parquet as pq

def load_data():
    """Загрузка данных"""
    train_data = pq.read_table('/kaggle/input/vseros-recsys/train_data.pq').to_pandas()
    sample_submission = pd.read_csv('/kaggle/input/vseros-recsys/sample_submission.csv')
    return train_data, sample_submission

def prepare_features(df):
    """Подготовка фичей и вычисление популярности"""
    # Находим последний день в данных
    max_date = df['date'].max()
    
    # Популярность товаров в последний день
    last_day_popularity = df[df['date'] == max_date]['item_id'].value_counts()
    
    # Общая популярность товаров
    global_popularity = df['item_id'].value_counts()
    
    return max_date, last_day_popularity, global_popularity

def generate_recommendations(train_data, sample_submission, days_lookback=5, top_n=20):
    """Генерация рекомендаций"""
    
    max_date, last_day_popularity, global_popularity = prepare_features(train_data)
    
    # Собираем последние взаимодействия пользователей
    user_recent_items = defaultdict(list)
    
    # Фильтруем данные за последние days_lookback дней
    recent_data = train_data[train_data['date'] >= (max_date - days_lookback + 1)]
    
    # Группируем по пользователям и собираем товары в порядке дат (от новых к старым)
    for user_id, group in recent_data.groupby('user_id'):
        items = group.sort_values('date', ascending=False)['item_id'].tolist()
        user_recent_items[user_id] = items
    
    # Готовим сабмишн
    submission_data = []
    
    for user_id in tqdm(sample_submission['user_id'].unique()):
        recommendations = []
        
        # Берем последние товары пользователя (уникальные, сохраняя порядок)
        if user_id in user_recent_items:
            recent_items = []
            seen = set()
            for item in user_recent_items[user_id]:
                if item not in seen:
                    recent_items.append(item)
                    seen.add(item)
                if len(recent_items) >= top_n:
                    break
            recommendations.extend(recent_items)
        
        # Если недостаточно рекомендаций, дополняем популярными товарами
        if len(recommendations) < top_n:
            remaining_slots = top_n - len(recommendations)
            
            # Берем популярные товары из последнего дня, исключая уже рекомендованные
            popular_items = []
            seen_recommendations = set(recommendations)
            
            for item in last_day_popularity.index:
                if item not in seen_recommendations:
                    popular_items.append(item)
                    if len(popular_items) >= remaining_slots:
                        break
            
            # Если все еще недостаточно, берем из общей популярности
            if len(popular_items) < remaining_slots:
                remaining_after_popular = remaining_slots - len(popular_items)
                for item in global_popularity.index:
                    if item not in seen_recommendations and item not in popular_items:
                        popular_items.append(item)
                        if len(popular_items) >= remaining_slots:
                            break
            
            recommendations.extend(popular_items[:remaining_slots])
        
        # Добавляем в сабмишн
        for item_id in recommendations[:top_n]:
            submission_data.append({'user_id': user_id, 'item_id': item_id})
    
    return pd.DataFrame(submission_data)

def main():
    """Основная функция"""
    print("Загрузка данных...")
    train_data, sample_submission = load_data()
    
    print("Генерация рекомендаций...")
    submission_df = generate_recommendations(train_data, sample_submission, days_lookback=18, top_n=20)
    
    print("Сохранение результата...")
    submission_df.to_csv('submission_18days-back.csv', index=False)
    
    print("Готово!")
    print(f"Сгенерировано рекомендаций для {len(submission_df['user_id'].unique())} пользователей")
    print(f"Всего рекомендаций: {len(submission_df)}")

if __name__ == "__main__":
    main()

Загрузка данных...
Генерация рекомендаций...


  0%|          | 0/293230 [00:00<?, ?it/s]

Сохранение результата...
Готово!
Сгенерировано рекомендаций для 293230 пользователей
Всего рекомендаций: 5864600


In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pyarrow.parquet as pq
from tqdm.auto import tqdm

def load_data():
    """Загрузка данных"""
    train_data = pq.read_table('/kaggle/input/vseros-recsys/train_data.pq').to_pandas()
    sample_submission = pd.read_csv('/kaggle/input/vseros-recsys/sample_submission.csv')
    return train_data, sample_submission

def prepare_features(df):
    """Подготовка фичей и вычисление популярности"""
    # Находим последний день в данных
    max_date = df['date'].max()
    
    # Товары, которые были активны в последнюю неделю
    last_week_items = set(df[df['date'] >= (max_date - 3)]['item_id'].unique())
    
    # Популярность товаров в последний день (только те, что были в последнюю неделю)
    last_day_data = df[df['date'] == max_date]
    last_day_popularity = last_day_data['item_id'].value_counts()
    # Фильтруем только товары из последней недели
    last_day_popularity = last_day_popularity[last_day_popularity.index.isin(last_week_items)]
    
    # Общая популярность товаров (только те, что были в последнюю неделю)
    global_popularity = df['item_id'].value_counts()
    global_popularity = global_popularity[global_popularity.index.isin(last_week_items)]
    
    # Популярность товаров за последнюю неделю (как альтернативный вариант)
    last_week_popularity = df[df['date'] >= (max_date - 6)]['item_id'].value_counts()
    
    return max_date, last_day_popularity, global_popularity, last_week_popularity, last_week_items

def generate_recommendations(train_data, sample_submission, days_lookback=18, top_n=20):
    """Генерация рекомендаций с учетом актуальности товаров"""
    
    max_date, last_day_popularity, global_popularity, last_week_popularity, last_week_items = prepare_features(train_data)
    
    print(f"Всего товаров в последнюю неделю: {len(last_week_items)}")
    print(f"Популярных товаров в последний день: {len(last_day_popularity)}")
    
    # Собираем последние взаимодействия пользователей
    user_recent_items = defaultdict(list)
    
    # Фильтруем данные за последние days_lookback дней
    recent_data = train_data[train_data['date'] >= (max_date - days_lookback + 1)]
    
    # Группируем по пользователям и собираем товары в порядке дат (от новых к старым)
    for user_id, group in tqdm(recent_data.groupby('user_id')):
        items = group.sort_values('date', ascending=False)['item_id'].tolist()
        user_recent_items[user_id] = items
    
    # Готовим сабмишн
    submission_data = []
    
    for user_id in tqdm(sample_submission['user_id'].unique()):
        recommendations = []
        
        # Берем последние товары пользователя (уникальные, сохраняя порядок)
        if user_id in user_recent_items:
            recent_items = []
            seen = set()
            for item in user_recent_items[user_id]:
                # Пропускаем товары, которых не было в последнюю неделю
                if item not in last_week_items:
                    continue
                if item not in seen:
                    recent_items.append(item)
                    seen.add(item)
                if len(recent_items) >= top_n:
                    break
            recommendations.extend(recent_items)
        
        # Если недостаточно рекомендаций, дополняем популярными товарами из последней недели
        if len(recommendations) < top_n:
            remaining_slots = top_n - len(recommendations)
            
            # Берем популярные товары из последнего дня, исключая уже рекомендованные
            popular_items = []
            seen_recommendations = set(recommendations)
            
            for item in last_day_popularity.index:
                if item not in seen_recommendations:
                    popular_items.append(item)
                    if len(popular_items) >= remaining_slots:
                        break
            
            # Если все еще недостаточно, берем из популярности за последнюю неделю
            if len(popular_items) < remaining_slots:
                remaining_after_popular = remaining_slots - len(popular_items)
                for item in last_week_popularity.index:
                    if item not in seen_recommendations and item not in popular_items:
                        popular_items.append(item)
                        if len(popular_items) >= remaining_slots:
                            break
            
            # Если все еще недостаточно, берем из общей популярности (только товары из последней недели)
            if len(popular_items) < remaining_slots:
                remaining_after_popular = remaining_slots - len(popular_items)
                for item in global_popularity.index:
                    if item not in seen_recommendations and item not in popular_items:
                        popular_items.append(item)
                        if len(popular_items) >= remaining_slots:
                            break
            
            recommendations.extend(popular_items[:remaining_slots])
        
        # Если все еще недостаточно (маловероятно), используем любые товары из последней недели
        if len(recommendations) < top_n:
            remaining_slots = top_n - len(recommendations)
            all_last_week_items = list(last_week_items - set(recommendations))
            # Берем случайные, но можно отсортировать по какой-то метрике
            additional_items = all_last_week_items[:remaining_slots]
            recommendations.extend(additional_items)
        
        # Добавляем в сабмишн
        for item_id in recommendations[:top_n]:
            submission_data.append({'user_id': user_id, 'item_id': item_id})
    
    return pd.DataFrame(submission_data)

def main():
    """Основная функция"""
    print("Загрузка данных...")
    train_data, sample_submission = load_data()
    
    print("Генерация рекомендаций...")
    submission_df = generate_recommendations(train_data, sample_submission, days_lookback=18, top_n=20)
    
    print("Сохранение результата...")
    submission_df.to_csv('submission_18days_back_filtered-3days.csv', index=False)
    
    print("Готово!")
    print(f"Сгенерировано рекомендаций для {len(submission_df['user_id'].unique())} пользователей")
    print(f"Всего рекомендаций: {len(submission_df)}")
    
    # Проверяем, что все рекомендованные товары были в последнюю неделю
    max_date = train_data['date'].max()
    last_week_items = set(train_data[train_data['date'] >= (max_date - 3)]['item_id'].unique())
    recommended_items = set(submission_df['item_id'].unique())
    
    print(f"Уникальных товаров в рекомендациях: {len(recommended_items)}")
    print(f"Из них были в последнюю неделю: {len(recommended_items & last_week_items)}")
    print(f"Неактуальных товаров: {len(recommended_items - last_week_items)}")

if __name__ == "__main__":
    main()

Загрузка данных...
Генерация рекомендаций...
Всего товаров в последнюю неделю: 162930
Популярных товаров в последний день: 56659


  0%|          | 0/1329348 [00:00<?, ?it/s]

  0%|          | 0/293230 [00:00<?, ?it/s]

Сохранение результата...
Готово!
Сгенерировано рекомендаций для 293230 пользователей
Всего рекомендаций: 5864600
Уникальных товаров в рекомендациях: 91535
Из них были в последнюю неделю: 91535
Неактуальных товаров: 0
